In [2]:
import netCDF4 as nc
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import math

In [3]:
data = Dataset("D:/HD climatenetwork/PAPER/filtered_t2m_1970_2022_5grid.nc", 'r')

# print variables 
for var in data.variables:
    print(var)

lat = data.variables['lat']
lon = data.variables['lon']
t2m = data.variables['t2m']

lat
lon
t2m


In [4]:
print(t2m)

<class 'netCDF4._netCDF4.Variable'>
float32 t2m(time, lat, lon)
unlimited dimensions: 
current shape = (19358, 37, 72)
filling on, default _FillValue of 9.969209968386869e+36 used
